# Pyradiomics feature calculation tool


<b>Description:</b><br>
Input:   Dicom directory path., pyradiomics parameters file (yaml) 
<br> Output:  Calculated features or NRRDs of image and mask
<br> <b>RTSTRUCT file should ends with '.dcm'</b>


In [2]:
########################
#@author: S.Primakov
#Utility for batch features extraction from dicom directories (pyradiomics, py_rex),
#Utility for batch DCM to Nrrd conversion
import General_func
import os
import time
import pandas as pd
from pandas import DataFrame


## Set the input data paths

convert_to_nrrd = True        #Set True if you want to convert DICOMs to Nrrds without features calculation
general_path = r'B:\Histology\Stanford\NSCLC Radiogenomics'    #Path to Dicom directory with patients
exportDir  = r'B:\NSCLC Radiogenomics stanford'    #Path for output xlsx file, or nrrd files
export_name = r'Maastro_Lung_features.xlsx'  #Name of output xlsx file with features
Region_of_interest = r'all'      #Set the ROI,  example 'gtv','ctv', etc, 'all' means all ROIs, start from ! if you want to extract only exactly mentioned ROI (example: if you write "gtv" it will extract all "gtv" contained ROIs, gtv2,gtv-t3 etc.) 
paramsFile =r'C:\Desktop\out\params.yaml' #path to pyradiomics parameters file
logg_enable= False              #Pyradiomics logg, if True txt logg file will be generated

In [ ]:
###Directory parser module
init_time = time.time()

# searching for DICOM files
patients = next(os.walk(general_path))[1]
print(len(patients),' Patients found')
Patient_dict={}

#loop over patients for finding RTSTRUCT
for patient in General_func.log_progress(patients,every=1):
    Patient_dict = General_func.Parsing_dir(general_path,patient,Patient_dict) 
    
print('Parsing time: %d sec'%(time.time()-init_time))
    

In [ ]:
start_time = time.time()     
Output_dictionary ={}
on=0

for patient in General_func.log_progress(Patient_dict.keys(),every=1): 
    Output_dictionary,on= General_func.Batch_calculation(Patient_dict,patient,Region_of_interest,convert_to_nrrd,exportDir,paramsFile,logg_enable,Output_dictionary,on)            
if not(convert_to_nrrd):
    Output_patients = DataFrame.from_dict(Output_dictionary).T
    print('Features shape : ',Output_patients.shape)
    writer = pd.ExcelWriter(exportDir+'/'+export_name)
    Output_patients.to_excel(writer,'Sheet1')
    writer.save()
    print('Calculation completed in %d seconds ' % (time.time() - start_time))
    print('Features file in directiry: %s'%exportDir)
else:           
    print('Conversion completed in %d seconds ' % (time.time() - start_time))
    print('Conerted files in directory: %s'%exportDir+'/converted_nrrds')

## Extra functions:


In [6]:
# Use this function to rename your dataset in case of it not ends with '.dcm'
import glob, os
def rename(direct, pattern, titlePattern):
    for pathAndFilename in glob.iglob(os.path.join(direct, pattern)):
        title, ext = os.path.splitext(os.path.basename(pathAndFilename))
        os.rename(pathAndFilename, 
                  os.path.join(direct, titlePattern % title + ext))

        
patients = next(os.walk(r'B:\Sebastian\For Sergey\For Sergey'))[1]
for patient in patients:
    rename(r'B:\Sebastian\For Sergey\For Sergey\%s'%patient, '*', '%s'+'.dcm')


In [5]:
#Patient dictionary writer function
import csv
dictionary_path=r'C:\Users\S.Primakov\Documents\Python Scripts\ucsfdict.csv'
with open(dictionary_path,'w') as dict_file:
    writer = csv.writer(dict_file)
    for key,value in Patient_dict.items():
        writer.writerow([key,value[0],value[1]])

In [ ]:
#Patient dictionary reader function
Patient_dict={}
import csv
dictionary_path=r'C:\Users\S.Primakov\Documents\Python Scripts\ucsfdict.csv'
with open(dictionary_path,'r') as dict_file:
    reader = csv.reader(dict_file)
    for i in reader:
        Patient_dict[i[0]]=[i[1],i[2]]

In [ ]:
####debug

In [5]:
import radiomics
from radiomics import featureextractor

In [6]:
radiomics.__version__

'1.3.0.post59+g2e6d2c1'